In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
!pip install bayesian-optimization

In [3]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv1D, Flatten
import tensorflow as tf
import numpy as np
import os
import random

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
from sklearn.preprocessing import OneHotEncoder

from bayes_opt import BayesianOptimization

In [4]:

train = pd.read_csv('./train_data.csv', encoding='cp949')
test = pd.read_csv('./test_data.csv', encoding='cp949')

## 전처리

In [5]:
#결측치 확인
def shownull(df):
    global missval_name
    print(f"데이터셋 차원 확인 \n {df.shape}")
    missval = df.isnull().sum()[df.isnull().sum()!=0]
    print(f"결측값 있는 변수 확인 \n {missval}")
    missval_name = missval.index # 결측치가 존재하는 열이 저장된 변수

#이상치 확인
def zscore_out(df, threshold = 3):
    mean = np.mean(df)
    std = np.std(df)
    z_score = np.abs((df - df.mean()) / df.std())
    outliers = np.where(z_score > threshold)
    # outlier_idx = outliers.nonzero()[0]
    print(outliers[0])

#시드고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(84)

### 결측치 대체

In [9]:
# Train set에 대해서 시간 변수 결측치 대체

train['regdate'] = pd.to_datetime(train['regdate'])

# 시작일과 종료일 설정
start_date = train['regdate'].min()  # 시작일: 2023-01-01 00:00
end_date = train['regdate'].max()  # 종료일: 2023-01-27 23:59    -------> min, max를 이용해서 data leakage 문제를 해결

# 결측치를 수정하고 이전 시간으로부터 순서대로 채워넣음
current_date = start_date  # 현재 날짜 초기화

for i in range(len(train)):
    # 결측치인 경우 이전 시간으로 대체
    if pd.isnull(train.loc[i, 'regdate']):
        train.loc[i, 'regdate'] = current_date
    
    # 다음 시간으로 이동
    current_date += timedelta(minutes=1)
    
    # 종료일인 경우 시작일로 되돌아감
    if current_date > end_date:
        current_date = start_date

#제대로 작동됨

In [10]:
# Test set에 대해서 시간 변수 결측치 대체

test['regdate'] = pd.to_datetime(test['regdate'])

# 시작일과 종료일 설정
start_date = test['regdate'].min()  # 시작일: 2023-01-01 00:00
end_date = test['regdate'].max()  # 종료일: 2023-01-27 23:59    -------> min, max를 이용해서 data leakage 문제를 해결

# 결측치를 수정하고 이전 시간으로부터 순서대로 채워넣음
current_date = start_date  # 현재 날짜 초기화

for i in range(len(test)):
    # 결측치인 경우 이전 시간으로 대체
    if pd.isnull(test.loc[i, 'regdate']):
        test.loc[i, 'regdate'] = current_date
    
    # 다음 시간으로 이동
    current_date += timedelta(minutes=1)
    
    # 종료일인 경우 시작일로 되돌아감
    if current_date > end_date:
        current_date = start_date

### 독립변수 추가

In [11]:
# TRAIN 요일 변수 추가
daysofweek = []
for i in range(len(train)) :
    dayweek = train.loc[i, 'regdate'].dayofweek
    daysofweek.append(dayweek)

train['dayofweek'] = daysofweek



#월요일 0 ~ 일요일6

<!--  -->

In [12]:
# TEST 요일 변수 추가

daysofweek = []
for i in range(len(test)) :
    dayweek = test.loc[i, 'regdate'].dayofweek
    daysofweek.append(dayweek)

test['dayofweek'] = daysofweek

In [13]:
# TRAIN 시간 -> 10진법 변환
train['time'] = train['regdate'].dt.strftime('%H:%M').str.split().str[0]
train['time'] = train['time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

In [14]:
# TEST 시간 -> 10진법 변환
test['time'] = test['regdate'].dt.strftime('%H:%M').str.split().str[0]
test['time'] = test['time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

In [15]:
it_is_not_na = np.where(~train[['PIR']].isnull())[0]

train.loc[it_is_not_na,['PIR']] = train.loc[it_is_not_na,['PIR']].astype(np.int64)

### 결측치 대체

In [16]:
# def cat_num_na(df) :

#     #변수 타입별로 분리
#     cat_na = df.select_dtypes(include=['object', 'int64'])
#     num_na = df.select_dtypes(include=['float64'])

#     #범주형 변수 최빈값으로 변환
#     for col in cat_na:
#         df[col].fillna(df[col].mode()[0], inplace=True)

#     #수치형 변수 중위값 대체
#     for col2 in num_na:
#         df[col2].fillna(df[col2].median(), inplace=True)

In [17]:
def make_area(df) :
    # 입력된 날짜 변수의 차이 계산
    date_range = df['regdate'].max() - df['regdate'].min()

    # 간격당 행의 개수 계산
    lowsbydate = ((date_range.days + 1) * 1440) #하루 1440분
    group_num = len(train) / lowsbydate

    for i in range(int(group_num)):
        start_index = i * lowsbydate
        end_index = (i + 1) * lowsbydate
        df.loc[start_index : end_index, 'area'] = i + 1

def inter_by_area(df):
    df.update(df.groupby('area').transform(lambda x: x.interpolate(method = 'pad')))
    return df

In [18]:
shownull(train)

데이터셋 차원 확인 
 (388800, 14)
결측값 있는 변수 확인 
 temp           78058
humi           78058
co2            78058
dust_pm1       78058
dust_pm_25     78058
dust_pm_10     78058
illuminance    78058
voc            78058
eco2           78058
PIR            78058
dtype: int64


In [21]:
shownull(train_nona)

데이터셋 차원 확인 
 (388800, 15)
결측값 있는 변수 확인 
 Series([], dtype: int64)


### 파일 분할 및 스케일링

In [22]:
#One-Hot Encoding
# ohe = OneHotEncoder(sparse=False)
# onehot_train = ohe.fit_transform(train_nona[['dayofweek']])
# onehot_test = ohe.transform(test_nona[['dayofweek']])

# onehot_train2 = pd.DataFrame(onehot_train, columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
# onehot_test2 = pd.DataFrame(onehot_test, columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

# onehot_train3 = pd.concat([train_nona.drop('dayofweek', axis=1), onehot_train2], axis=1)
# onehot_test3 = pd.concat([test_nona.drop('dayofweek', axis=1), onehot_test2], axis=1)

In [340]:
train_nona = train.sort_values(by=["time","regdate"], ascending=[True,True]) 
test_nona = test.sort_values(by=["time","regdate"], ascending=[True,True]) 
train_nona=train.reset_index(drop=True)
test_nona=test.reset_index(drop=True)

In [341]:
X_train = train_nona.drop('재실인원', axis=1)
y_train = train_nona['재실인원']

X_test = test_nona.drop('재실인원', axis=1)
y_test = test_nona['재실인원']

In [342]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [343]:
#regdate 제거한 뒤 확인

X_train = X_train.drop(['regdate','area','dayofweek'], axis=1)
X_valid = X_valid.drop(['regdate','area','dayofweek'], axis=1)
X_test = X_test.drop(['regdate','area','dayofweek'], axis=1)

In [344]:
X_train.sort_index(inplace=True)
X_valid.sort_index(inplace=True)
X_test.sort_index(inplace=True)
y_train.sort_index(inplace=True)
y_valid.sort_index(inplace=True)
y_test.sort_index(inplace=True)

In [345]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [346]:
y_train = np.array(y_train)
y_valid = np.array(y_valid)
y_test = np.array(y_test)

In [347]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,10
0,0.488550,0.302632,0.173448,0.021739,0.014706,0.007564,0.000000,0.010819,0.048752,0.0,0.0
1,0.366412,0.210526,0.152463,0.030435,0.020588,0.010590,0.007194,0.000763,0.010288,0.0,0.0
2,0.572519,0.302632,0.164454,0.020290,0.013725,0.007060,0.000000,0.038682,0.075538,0.0,0.0
3,0.770992,0.289474,0.376445,0.020290,0.013725,0.007060,0.043165,0.088823,0.123301,0.0,0.0
4,0.458015,0.355263,0.211135,0.021739,0.014706,0.007564,0.035971,0.063920,0.099883,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
291595,0.465649,0.197368,0.331906,0.010145,0.006863,0.003530,0.071942,0.000381,0.005252,0.0,1.0
291596,0.450382,0.144737,0.214561,0.010145,0.006863,0.003530,0.071942,0.002762,0.034448,0.0,1.0
291597,0.328244,0.144737,0.158887,0.005797,0.003922,0.002017,0.007194,0.000046,0.000680,0.0,1.0
291598,0.641221,0.144737,0.365739,0.007246,0.004902,0.002521,0.079137,0.000290,0.003924,0.0,1.0


In [348]:
X_train=X_train.reshape(-1,2,11)
X_test=X_test.reshape(-1,2,11)
X_valid=X_valid.reshape(-1,2,11)

In [352]:
y_train=y_train.reshape(-1,2)
y_test=y_test.reshape(-1,2)
y_valid=y_valid.reshape(-1,2)

In [357]:
loss, mse =model4.evaluate(X_test, y_test)
mse

1329/1329 [==============================] - 5s 4ms/step - loss: 6.4637 - mse: 6.4637


6.463665962219238

In [358]:
mse =[]
def buildModel(neurons,drop,nb_batch,hidden_layers):  
    nb_epoch=5
    neurons=int(neurons)
    hidden_layers=int(hidden_layers)

    # input layer
    inputs = tf.keras.Input(shape=(2,11))
    x = tf.keras.layers.GRU(neurons*(2**(hidden_layers+1)), return_sequences=True, recurrent_dropout=0.2, activation='relu')(inputs)  

    if hidden_layers !=0:
        for i in range(1,hidden_layers+1):
            x = tf.keras.layers.Dense(neurons*(2**(hidden_layers+1)), activation='relu')(x)  
    x = tf.keras.layers.Dropout(drop)(x)
    

    # output
    predictions = Dense(1)(x)


    # modeling
    model = tf.keras.Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode="min",patience=3,restore_best_weights=True, verbose=1)



    # learning
    model.fit(X_train, y_train, epochs=int(nb_epoch), batch_size=int(nb_batch),callbacks=[early_stopping], validation_data=(X_valid, y_valid))


    # evaluate the model with test data
    loss, mse =model.evaluate(X_test, y_test)
    
    if mse < 1: 
        model.save(f"./model/rmse/{los_test}.h5")
        print("모델이 저장 되었습니다.")
        print("mse : ",mse)
    
    return mse

In [359]:
pbounds = {
        # define the range of each parameter
        'drop' : (0.5,0.9),
        "neurons" : (12,24),
        'nb_batch' : (512,1024),
        'hidden_layers' : (0,3)}

In [360]:
bo=BayesianOptimization(f=buildModel, pbounds=pbounds, verbose=2, random_state=42 )

In [361]:
bo.maximize(
   init_points=2,
   n_iter= 100
)

|   iter    |  target   |   drop    | hidden... | nb_batch  |  neurons  |
-------------------------------------------------------------------------
Epoch 1/5
107/107 [==============================] - 5s 31ms/step - loss: 1.8205 - mean_squared_error: 1.8205 - val_loss: 1.1309 - val_mean_squared_error: 1.1309
Epoch 2/5
107/107 [==============================] - 3s 32ms/step - loss: 1.0072 - mean_squared_error: 1.0072 - val_loss: 0.8616 - val_mean_squared_error: 0.8616
Epoch 3/5
107/107 [==============================] - 3s 32ms/step - loss: 0.8332 - mean_squared_error: 0.8332 - val_loss: 0.7845 - val_mean_squared_error: 0.7845
Epoch 4/5
107/107 [==============================] - 4s 33ms/step - loss: 0.7674 - mean_squared_error: 0.7674 - val_loss: 0.7252 - val_mean_squared_error: 0.7252
Epoch 5/5
1329/1329 [==============================] - 11s 8ms/step - loss: 3.7948 - mean_squared_error: 3.7948
| 1         | 3.795     | 0.5873    | 2.852     | 1.366e+03 | 15.99     |
Epoch 1/5
142/142 

Epoch 4/5
145/145 [==============================] - 5s 34ms/step - loss: 0.7400 - mean_squared_error: 0.7400 - val_loss: 0.7113 - val_mean_squared_error: 0.7113
Epoch 5/5
1329/1329 [==============================] - 10s 8ms/step - loss: 43.1836 - mean_squared_error: 43.1836
| 14        | 43.18     | 0.5031    | 2.91      | 1.007e+03 | 10.69     |
Epoch 1/5
143/143 [==============================] - 7s 35ms/step - loss: 2.6811 - mean_squared_error: 2.6811 - val_loss: 1.9825 - val_mean_squared_error: 1.9825
Epoch 2/5
143/143 [==============================] - 5s 33ms/step - loss: 1.9229 - mean_squared_error: 1.9229 - val_loss: 1.6250 - val_mean_squared_error: 1.6250
Epoch 3/5
143/143 [==============================] - 5s 34ms/step - loss: 1.6269 - mean_squared_error: 1.6269 - val_loss: 1.3930 - val_mean_squared_error: 1.3930
Epoch 4/5
143/143 [==============================] - 5s 33ms/step - loss: 1.4271 - mean_squared_error: 1.4271 - val_loss: 1.2124 - val_mean_squared_error: 1.2124
Ep

Epoch 3/5
109/109 [==============================] - 3s 31ms/step - loss: 0.8078 - mean_squared_error: 0.8078 - val_loss: 0.7921 - val_mean_squared_error: 0.7921
Epoch 4/5
109/109 [==============================] - 4s 32ms/step - loss: 0.7357 - mean_squared_error: 0.7357 - val_loss: 0.7330 - val_mean_squared_error: 0.7330
Epoch 5/5
1329/1329 [==============================] - 10s 8ms/step - loss: 35.1750 - mean_squared_error: 35.1750
| 21        | 35.18     | 0.4422    | 2.429     | 1.346e+03 | 16.05     |
Epoch 1/5
107/107 [==============================] - 6s 36ms/step - loss: 1.9152 - mean_squared_error: 1.9152 - val_loss: 1.1526 - val_mean_squared_error: 1.1526
Epoch 2/5
107/107 [==============================] - 3s 32ms/step - loss: 1.2100 - mean_squared_error: 1.2100 - val_loss: 0.8883 - val_mean_squared_error: 0.8883
Epoch 3/5
107/107 [==============================] - 4s 33ms/step - loss: 1.0250 - mean_squared_error: 1.0250 - val_loss: 0.8152 - val_mean_squared_error: 0.8152
Ep

Epoch 2/5
110/110 [==============================] - 3s 31ms/step - loss: 1.9492 - mean_squared_error: 1.9492 - val_loss: 1.7070 - val_mean_squared_error: 1.7070
Epoch 3/5
110/110 [==============================] - 3s 32ms/step - loss: 1.7487 - mean_squared_error: 1.7487 - val_loss: 1.5407 - val_mean_squared_error: 1.5407
Epoch 4/5
110/110 [==============================] - 3s 32ms/step - loss: 1.5849 - mean_squared_error: 1.5849 - val_loss: 1.3936 - val_mean_squared_error: 1.3936
Epoch 5/5
1329/1329 [==============================] - 9s 6ms/step - loss: 17.5297 - mean_squared_error: 17.5297
| 28        | 17.53     | 0.5737    | 0.6522    | 1.333e+03 | 11.6      |
Epoch 1/5
127/127 [==============================] - 6s 35ms/step - loss: 2.0553 - mean_squared_error: 2.0553 - val_loss: 1.3694 - val_mean_squared_error: 1.3694
Epoch 2/5
127/127 [==============================] - 4s 32ms/step - loss: 1.2057 - mean_squared_error: 1.2057 - val_loss: 1.0773 - val_mean_squared_error: 1.0773
Epo

Epoch 1/5
114/114 [==============================] - 6s 36ms/step - loss: 2.7310 - mean_squared_error: 2.7310 - val_loss: 2.0288 - val_mean_squared_error: 2.0288
Epoch 2/5
114/114 [==============================] - 4s 33ms/step - loss: 2.0185 - mean_squared_error: 2.0185 - val_loss: 1.7006 - val_mean_squared_error: 1.7006
Epoch 3/5
114/114 [==============================] - 4s 33ms/step - loss: 1.7600 - mean_squared_error: 1.7600 - val_loss: 1.5044 - val_mean_squared_error: 1.5044
Epoch 4/5
114/114 [==============================] - 4s 32ms/step - loss: 1.5834 - mean_squared_error: 1.5834 - val_loss: 1.3511 - val_mean_squared_error: 1.3511
Epoch 5/5
1329/1329 [==============================] - 9s 7ms/step - loss: 16.2951 - mean_squared_error: 16.2951
| 35        | 16.3      | 0.7003    | 0.5213    | 1.287e+03 | 17.85     |
Epoch 1/5
142/142 [==============================] - 7s 34ms/step - loss: 2.6360 - mean_squared_error: 2.6360 - val_loss: 1.9267 - val_mean_squared_error: 1.9267
Epo

...

Epoch 4/5
98/98 [==============================] - 3s 33ms/step - loss: 1.5795 - mean_squared_error: 1.5795 - val_loss: 1.4214 - val_mean_squared_error: 1.4214
Epoch 5/5
1329/1329 [==============================] - 9s 7ms/step - loss: 14.3230 - mean_squared_error: 14.3230
| 69        | 14.32     | 0.5743    | 0.213     | 1.49e+03  | 15.62     |
Epoch 1/5
122/122 [==============================] - 6s 35ms/step - loss: 1.8594 - mean_squared_error: 1.8594 - val_loss: 1.1261 - val_mean_squared_error: 1.1261
Epoch 2/5
122/122 [==============================] - 4s 34ms/step - loss: 1.0397 - mean_squared_error: 1.0397 - val_loss: 0.8763 - val_mean_squared_error: 0.8763
Epoch 3/5
122/122 [==============================] - 4s 34ms/step - loss: 0.8682 - mean_squared_error: 0.8682 - val_loss: 0.7905 - val_mean_squared_error: 0.7905
Epoch 4/5
122/122 [==============================] - 4s 33ms/step - loss: 0.7972 - mean_squared_error: 0.7972 - val_loss: 0.7556 - val_mean_squared_error: 0.7556
Epoch

Epoch 3/5
110/110 [==============================] - 4s 33ms/step - loss: 1.2000 - mean_squared_error: 1.2000 - val_loss: 0.9943 - val_mean_squared_error: 0.9943
Epoch 4/5
110/110 [==============================] - 4s 32ms/step - loss: 1.0975 - mean_squared_error: 1.0975 - val_loss: 0.9313 - val_mean_squared_error: 0.9313
Epoch 5/5
1329/1329 [==============================] - 10s 7ms/step - loss: 1.7249 - mean_squared_error: 1.7249
| 76        | 1.725     | 0.7444    | 1.811     | 1.334e+03 | 15.82     |
Epoch 1/5
135/135 [==============================] - 6s 35ms/step - loss: 2.6794 - mean_squared_error: 2.6794 - val_loss: 1.9475 - val_mean_squared_error: 1.9475
Epoch 2/5
135/135 [==============================] - 4s 32ms/step - loss: 2.1014 - mean_squared_error: 2.1014 - val_loss: 1.6826 - val_mean_squared_error: 1.6826
Epoch 3/5
135/135 [==============================] - 4s 32ms/step - loss: 1.8873 - mean_squared_error: 1.8873 - val_loss: 1.5060 - val_mean_squared_error: 1.5060
Epoc

Epoch 2/5
123/123 [==============================] - 4s 36ms/step - loss: 1.4908 - mean_squared_error: 1.4908 - val_loss: 1.1255 - val_mean_squared_error: 1.1255
Epoch 3/5
123/123 [==============================] - 4s 36ms/step - loss: 1.3294 - mean_squared_error: 1.3294 - val_loss: 1.0218 - val_mean_squared_error: 1.0218
Epoch 4/5
123/123 [==============================] - 4s 36ms/step - loss: 1.2251 - mean_squared_error: 1.2251 - val_loss: 0.9393 - val_mean_squared_error: 0.9393
Epoch 5/5
1329/1329 [==============================] - 10s 7ms/step - loss: 8.2745 - mean_squared_error: 8.2745
| 83        | 8.274     | 0.8201    | 1.92      | 1.194e+03 | 13.04     |
Epoch 1/5
121/121 [==============================] - 6s 36ms/step - loss: 1.7957 - mean_squared_error: 1.7957 - val_loss: 1.0842 - val_mean_squared_error: 1.0842
Epoch 2/5
121/121 [==============================] - 4s 32ms/step - loss: 1.0447 - mean_squared_error: 1.0447 - val_loss: 0.8431 - val_mean_squared_error: 0.8431
Epoc

Epoch 1/5
116/116 [==============================] - 6s 35ms/step - loss: 1.6885 - mean_squared_error: 1.6885 - val_loss: 1.0596 - val_mean_squared_error: 1.0596
Epoch 2/5
116/116 [==============================] - 4s 33ms/step - loss: 0.9381 - mean_squared_error: 0.9381 - val_loss: 0.8311 - val_mean_squared_error: 0.8311
Epoch 3/5
116/116 [==============================] - 4s 31ms/step - loss: 0.8007 - mean_squared_error: 0.8007 - val_loss: 0.7750 - val_mean_squared_error: 0.7750
Epoch 4/5
116/116 [==============================] - 4s 33ms/step - loss: 0.7404 - mean_squared_error: 0.7404 - val_loss: 0.7169 - val_mean_squared_error: 0.7169
Epoch 5/5
1329/1329 [==============================] - 10s 8ms/step - loss: 35.5738 - mean_squared_error: 35.5738
| 90        | 35.57     | 0.5259    | 2.933     | 1.266e+03 | 16.55     |
Epoch 1/5
105/105 [==============================] - 6s 35ms/step - loss: 2.3100 - mean_squared_error: 2.3100 - val_loss: 1.5349 - val_mean_squared_error: 1.5349
Ep

Epoch 1/5
109/109 [==============================] - 6s 35ms/step - loss: 1.6367 - mean_squared_error: 1.6367 - val_loss: 1.0608 - val_mean_squared_error: 1.0608
Epoch 2/5
109/109 [==============================] - 3s 31ms/step - loss: 0.8941 - mean_squared_error: 0.8941 - val_loss: 0.8196 - val_mean_squared_error: 0.8196
Epoch 3/5
109/109 [==============================] - 3s 31ms/step - loss: 0.7535 - mean_squared_error: 0.7535 - val_loss: 0.7537 - val_mean_squared_error: 0.7537
Epoch 4/5
109/109 [==============================] - 3s 30ms/step - loss: 0.6999 - mean_squared_error: 0.6999 - val_loss: 0.7259 - val_mean_squared_error: 0.7259
Epoch 5/5
1329/1329 [==============================] - 10s 7ms/step - loss: 29.2995 - mean_squared_error: 29.2995
| 97        | 29.3      | 0.4367    | 2.926     | 1.34e+03  | 16.02     |
Epoch 1/5
109/109 [==============================] - 6s 35ms/step - loss: 1.6753 - mean_squared_error: 1.6753 - val_loss: 1.0786 - val_mean_squared_error: 1.0786
Ep

In [ ]:
reconstructed_model = keras.models.load_model("./model/0.9612308144569397.h5")

In [ ]:
Y_train_pred = reconstructed_model.predict(X_train)

In [ ]:
Y_test_pred = reconstructed_model.predict(X_test)

In [ ]:
print(X_test.shape,X_train.shape,y_test.shape,y_train.shape )

In [ ]:
y_test=y_test.reshape(-1,1)

In [ ]:
y_train=y_train.reshape(-1,1)

In [ ]:
import matplotlib.pyplot as plt
def evaluation(Y_real, Y_pred, graph_on=False):
    loss_length = len(Y_real.values.flatten()) - len(Y_pred)
    if loss_length != 0:
        Y_real = Y_real[loss_length:]
    if graph_on == True:
        pd.concat([Y_real, pd.DataFrame(Y_pred, index=Y_real.index, columns=['prediction'])], axis=1).plot(kind='line', figsize=(20,6),
                                                                                                           xlim=(Y_real.index.min(),Y_real.index.max()),
                                                                                                           linewidth=3, fontsize=20)
        plt.title('Time Series of Target', fontsize=20)
        plt.xlabel('Index', fontsize=15)
        plt.ylabel('Target Value', fontsize=15)
    MAE = abs(Y_real.values.flatten() - Y_pred).mean()
    MSE = ((Y_real.values.flatten() - Y_pred)**2).mean()
    MAPE = (abs(Y_real.values.flatten() - Y_pred)/Y_real.values.flatten()*100).mean()
    Score = pd.DataFrame([MAE, MSE, MAPE], index=['MAE', 'MSE', 'MAPE'], columns=['Score']).T
    Residual = pd.DataFrame(Y_real.values.flatten() - Y_pred, index=Y_real.index, columns=['Error'])
    return Score, Residual

def evaluation_trte(Y_real_tr, Y_pred_tr, Y_real_te, Y_pred_te, graph_on=False):
    Score_tr, Residual_tr = evaluation(Y_real_tr, Y_pred_tr, graph_on=graph_on)
    Score_te, Residual_te = evaluation(Y_real_te, Y_pred_te, graph_on=graph_on)
    Score_trte = pd.concat([Score_tr, Score_te], axis=0)
    Score_trte.index = ['Train', 'Test']
    return Score_trte, Residual_tr, Residual_te

In [ ]:
Score_GRU, Residual_tr, Residual_te = evaluation_trte(pd.DataFrame(y_train), Y_train_pred.flatten(), pd.DataFrame(y_test), Y_test_pred.flatten(), graph_on=True)

In [ ]:
display(Score_GRU)

In [ ]:
# nb_epoch=5
# neurons=3
# hidden_layers=3
# drop=0.5
# nb_batch=500

# # input layer
# inputs = tf.keras.Input(shape=(1,12))
# x = tf.keras.layers.GRU(neurons*(2**(hidden_layers+1)), return_sequences=True, recurrent_dropout=0.2, activation='relu')(inputs)  

# if hidden_layers !=0:
#     for i in range(1,hidden_layers+1):
#         x = tf.keras.layers.GRU(neurons*(2**(hidden_layers+1)), return_sequences=True, activation='relu')(x)
# x = tf.keras.layers.GRU(neurons*(2**(hidden_layers+1)),return_sequences=False, activation='relu')(inputs)        
# x = tf.keras.layers.Dropout(drop)(x)


# # output
# predictions = Dense(1)(x)


# # modeling
# model = tf.keras.Model(inputs=inputs, outputs=predictions)
# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode="min",patience=3,restore_best_weights=True, verbose=1)



# # learning
# model.fit(X_train, y_train, epochs=int(nb_epoch), batch_size=int(nb_batch),callbacks=[early_stopping], validation_data=(X_valid, y_valid))


In [ ]:
# a,b=model.evaluate(X_test, y_test)

In [ ]:
# los_test